In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Loading the dataset & Pre trained word embeddings

In [2]:
import os
import sys
import torch
import torchtext.data
import torchtext.datasets
import numpy as np 
import time
from RNN import SentimentGRU
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 

from cs236781.train_results import FitResult
from cs236781 import plot

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device =", device)

device = cuda


load the data set:

In [5]:
if os.sys.platform == 'linux':
    data_dir = os.path.expanduser('~/HW4/project/GloVe')
else:
    data_dir = os.path.expanduser('~/.pytorch-datasets')
    
# torchtext Field objects parse text (e.g. a review) and create a tensor representation

# This Field object will be used for tokenizing the movie reviews text
review_parser = torchtext.data.Field(
    sequential=True, use_vocab=True, lower=True,
    init_token='<sos>', eos_token='<eos>', dtype=torch.long,
    tokenize='spacy', tokenizer_language='en_core_web_sm'
)

# This Field object converts the text labels into numeric values (0,1,2)
label_parser = torchtext.data.Field(
    is_target=True, sequential=False, unk_token=None, use_vocab=True
)

# Load SST, tokenize the samples and labels
# ds_X are Dataset objects which will use the parsers to return tensors
ds_train, ds_valid, ds_test = torchtext.datasets.SST.splits(
    review_parser, label_parser, root=data_dir
)

n_train = len(ds_train)
print(f'Number of training   samples: {n_train}')
print(f'Number of validation samples: {len(ds_valid)}')
print(f'Number of test       samples: {len(ds_test)}')

/home/yuval.goshen/miniconda3/envs/cs236781-hw/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


downloading trainDevTestTrees_PTB.zip


trainDevTestTrees_PTB.zip: 100%|██████████| 790k/790k [00:03<00:00, 238kB/s]  
/home/yuval.goshen/miniconda3/envs/cs236781-hw/lib/python3.8/site-packages/torchtext/data/example.py:94: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/home/yuval.goshen/miniconda3/envs/cs236781-hw/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


extracting
Number of training   samples: 8544
Number of validation samples: 1101
Number of test       samples: 2210


As required, we'll use the pre-trained word embeddings of glove 6B.

In [8]:
#Vocabulary size is 40k, Embedding chosen size in 50
vocab, embeddings = [],[]
with open('./GloVe/glove.6B.50d.txt','rt',encoding='utf8') as fi:
    full_content = fi.read().strip().split('\n')
for i in range(len(full_content)):
    i_word = full_content[i].split(' ')[0]
    i_embeddings = [float(val) for val in full_content[i].split(' ')[1:]]
    vocab.append(i_word)
    embeddings.append(i_embeddings)
    

add padding and unknown tokens to the embeddings array:

In [9]:
# Add the padding and the unknown tokens to the vocab and embeddings arrays

vocab = np.array(vocab) 
embeddings = np.array(embeddings)
vocab = np.insert(vocab, 0, '<pad>')
vocab = np.insert(vocab, 1, '<unk>')

unk_emb = np.mean(embeddings, axis=0, keepdims=True)
pad_emb = np.zeros_like(embeddings[0]).reshape(1,-1)


embeddings = np.vstack((pad_emb, unk_emb, embeddings))

print(vocab[:10])

['<pad>' '<unk>' 'the' ',' '.' 'of' 'to' 'and' 'in' 'a']


## Baseline Model - Sentiment Analysis using RNN - GRU

As for the first part in our experiment


In [54]:
def train(model, optimizer, loss_fn, dl_train, dl_test, max_epochs=100,
          num_batches=400, save_path=None):
    
    best_test_acc = 0
    res = FitResult(max_epochs,[],[],[],[] )
    
    for epoch_idx in range(max_epochs):
        total_loss, num_correct = 0, 0
        num_samples = 0
        start_time = time.time()

        for batch_idx, batch in enumerate(dl_train):
            X, y = batch.text, batch.label

            # Forward pass
            _, y_pred_log_proba = model(X)

            # Backward pass
            optimizer.zero_grad()
            loss = loss_fn(y_pred_log_proba, y)
            loss.backward()

            # Weight updates
            optimizer.step()

            # Calculate accuracy
            total_loss += loss.item()
            y_pred = torch.argmax(y_pred_log_proba, dim=1)
            num_samples += y_pred.shape[0]
            num_correct += torch.sum(y_pred == y).float().item()

            if batch_idx == num_batches-1:
                break
        
        curr_test_loss, curr_test_acc = test_epoch(model, loss_fn, dl_test)
        res.test_loss.append(curr_test_loss)
        res.test_acc.append(curr_test_acc)
        
        curr_train_loss = total_loss /(num_batches)
        curr_train_acc = num_correct /(num_samples)
        res.train_loss.append(curr_train_loss)
        res.train_acc.append(curr_train_acc)
        
        
        print(f"Epoch #{epoch_idx}, loss={curr_train_loss:.3f}, accuracy={curr_train_acc:.3f}, elapsed={time.time()-start_time:.1f} sec")
        
        if save_path and curr_test_acc > best_test_acc:
            print("---saving model ---")
            torch.save(model, save_path)
            best_test_acc = curr_test_acc
            
    return res

In [55]:
def test_epoch(model, loss_fn, dataloader):
    model.eval()
    total_loss = 0
    num_correct = 0 
    num_batches = 0
    
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for batch_idx, batch in enumerate(dataloader):
            num_batches = batch_idx
            X, y = batch.text, batch.label

            _, y_test = model(X)
            loss = loss_fn(y_test, y)
            total_loss += loss.item()
            
            # Calculate accuracy
            y_pred = torch.argmax(y_test, dim=1)
            num_correct += torch.sum(y_pred == y).float().item()
            
        num_batches += 1   
        
        test_loss = total_loss /(num_batches)
        print("Test loss is ----", test_loss, "-----")
        test_acc = num_correct /(num_batches * BATCH_SIZE)

    model.train()
    return test_loss, test_acc

In [12]:
EMBEDDING_DIM = 50
BATCH_SIZE = 32
HIDDEN_SIZE = 256
NUM_LAYERS = 2
DROPOUT = 0
LEARNING_RATE = 1e-3

In [13]:
dl_train, dl_valid, dl_test = torchtext.data.BucketIterator.splits(
    (ds_train, ds_valid, ds_test), batch_size=BATCH_SIZE,
    shuffle=True, device=device)
review_parser.build_vocab(ds_train)
label_parser.build_vocab(ds_train)

/home/yuval.goshen/miniconda3/envs/cs236781-hw/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [52]:
# defining the baseline model
model = SentimentGRU(embeddings, hidden_size=HIDDEN_SIZE,
                     num_layers=NUM_LAYERS, dropout=DROPOUT).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

loss_fn = torch.nn.NLLLoss()


In [53]:
fit_res = train(model, optimizer, loss_fn, dl_train, dl_test, max_epochs=100,
      num_batches=500, save_path = "./models/sentimentGRU.pt")

Test loss is ---- 1.0426211527415685 -----
Epoch #0, loss=0.561, accuracy=0.894, elapsed=2.9 sec
---saving model ---
Test loss is ---- 1.0344108138765609 -----
Epoch #1, loss=0.560, accuracy=0.885, elapsed=2.8 sec
---saving model ---
Test loss is ---- 1.0380218386650086 -----
Epoch #2, loss=0.560, accuracy=0.884, elapsed=2.8 sec
Test loss is ---- 1.0329838420663562 -----
Epoch #3, loss=0.560, accuracy=0.890, elapsed=2.8 sec
---saving model ---
Test loss is ---- 1.0416357696056366 -----
Epoch #4, loss=0.560, accuracy=0.886, elapsed=2.8 sec


KeyboardInterrupt: 

In [45]:
plot.plot_fit(fit_res)

NameError: name 'fit_res' is not defined

In [59]:
# Load the best model achieved during training
model = torch.load("./models/sentimentGRU.pt")
print(model)

SentimentGRU(
  (embedding_layer): Embedding(400002, 50)
  (gru): GRU(50, 128, num_layers=2, dropout=0.5)
  (dense_linear): Linear(in_features=128, out_features=3, bias=True)
  (log_softmax): LogSoftmax(dim=1)
)


In [84]:
# Self Attenttion Hyper parameters:

EMBEDDING_DIM = 50
BATCH_SIZE = 8
NUM_HEADS = 1
D_MODEL = 50
TWO_LAYERS = False
DROPOUT = 0.7
LEARNING_RATE = 1e-3

In [85]:
from SelfAttention import SentimentSelfAttention

dl_train, dl_valid, dl_test = torchtext.data.BucketIterator.splits(
    (ds_train, ds_valid, ds_test), batch_size=BATCH_SIZE,
    shuffle=True, device=device)
review_parser.build_vocab(ds_train)
label_parser.build_vocab(ds_train)

model_attention = SentimentSelfAttention(embeddings,
                                         d_model=D_MODEL,
                                         num_heads=NUM_HEADS,
                                         dropout=DROPOUT,
                                         two_layers=TWO_LAYERS).to(device)

att_optimizer = torch.optim.Adam(model_attention.parameters(), lr=LEARNING_RATE)

print(model_attention)
print("trainable params:", 
      sum(p.numel() for p in model_attention.parameters() if p.requires_grad)) 


SentimentSelfAttention(
  (embedding_layer): Embedding(400002, 50)
  (PositionalEncoding): PositionalEncoding()
  (q_feedforward): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): Dropout(p=0.7, inplace=False)
    (2): ReLU()
  )
  (k_feedforward): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): Dropout(p=0.7, inplace=False)
    (2): ReLU()
  )
  (v_feedforward): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): Dropout(p=0.7, inplace=False)
    (2): ReLU()
  )
  (SelfAttention1): MultiheadAttention(
    (out_proj): _LinearWithBias(in_features=50, out_features=50, bias=True)
  )
  (attention_out_feedforward): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): Dropout(p=0.7, inplace=False)
    (2): ReLU()
  )
  (dense_linear): Linear(in_features=50, out_features=3, bias=True)
  (log_softmax): LogSoftmax(dim=1)
)
trainable params: 20553


In [86]:
Attention_res = train(model_attention, att_optimizer, loss_fn, dl_train, dl_test, max_epochs=100,
          num_batches=500, save_path="./models/selfAttention.pt")

Test loss is ---- 1.036583632983886 -----
Epoch #0, loss=1.062, accuracy=0.409, elapsed=19.1 sec
---saving model ---
Test loss is ---- 1.0404844544424476 -----
Epoch #1, loss=1.057, accuracy=0.423, elapsed=19.0 sec


KeyboardInterrupt: 

In [ ]:
plot.plot_fit(Attention_res)

In [71]:
# Load the best model achieved during training
model_attention = torch.load("./models/selfAttention.pt")
print(model_attention)

SentimentSelfAttention(
  (embedding_layer): Embedding(400002, 50)
  (PositionalEncoding): PositionalEncoding()
  (q_feedforward): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): Dropout(p=0, inplace=False)
    (2): ReLU()
  )
  (k_feedforward): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): Dropout(p=0, inplace=False)
    (2): ReLU()
  )
  (v_feedforward): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): Dropout(p=0, inplace=False)
    (2): ReLU()
  )
  (SelfAttention1): MultiheadAttention(
    (out_proj): _LinearWithBias(in_features=50, out_features=50, bias=True)
  )
  (attention_out_feedforward): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): Dropout(p=0, inplace=False)
    (2): ReLU()
  )
  (dense_linear): Linear(in_features=50, out_features=3, bias=True)
  (log_softmax): LogSoftmax(dim=1)
)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, ConfusionMatrixDisplay

# get the best predictions from the model
def get_preds_and_labels(model, dl, device):
    model.eval()
    all_preds = torch.tensor([]).to(device)
    GT_labels = torch.tensor([]).to(device)

    for i,batch in enumerate(dl):
        X, y = batch
        
        _, preds = model(X)
        preds = torch.argmax(preds, dim=1)
        
        
        all_preds = torch.cat((all_preds, preds), dim=0)
        GT_labels = torch.cat((GT_labels, y), dim=0)

    return all_preds , GT_labels

with torch.no_grad():
    # baseline VS SelfAttention Confusion matrices
    bl_best_preds, bl_GT_preds= get_preds_and_labels(model, dl_test, device)
    se_best_preds, se_GT_preds = get_preds_and_labels(model_attention, dl_test, device)
    
    # Now lets demonstrate the confusion matrix of the test set.
    IC = type('IdentityClassifier', (), {"predict": lambda i : i, "_estimator_type": "classifier"})
    class_names = ['Positive','Negative','Neutral']

    fig, (ax1, ax2) = plt.subplots(ncols = 2, nrows = 1 , figsize=(25, 10))
    disp = plot_confusion_matrix(IC, bl_GT_preds.cpu(), bl_best_preds.cpu() ,display_labels=class_names, cmap=plt.cm.Blues, normalize='true', ax=ax1);
    disp.ax_.set_title('Baseline Confusion Matrix\n')

    disp = plot_confusion_matrix(IC, se_GT_preds.cpu(), se_best_preds.cpu() ,display_labels=class_names, cmap=plt.cm.Blues, normalize='true', ax=ax2);
    disp.ax_.set_title('Self-Attention Confusion Matrix\n')

    

In [ ]:
# batch = next(iter(dl_train)).text.to(device)
# print("batch shape:", batch.shape)
# _ = model_attention(batch)